In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import matplotlib.pyplot as plt
import os
import glob
import skimage.io as io
import skimage.transform as trans
from model import *

Using TensorFlow backend.


# Color_Dict

In [2]:
Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

# Load Data

In [15]:
image_path = '/home/jasonyu/zsq/unet-master/unet-master/data/train/image/'
mask_path = '/home/jasonyu/zsq/unet-master/unet-master/data/train/label/'
test_path = '/home/jasonyu/zsq/unet-master/unet-master/data/test/'
result_path = '/home/jasonyu/zsq/unet-master/unet-master/result/'
image_prefix = 'image_'
mask_prefix  = 'mask_'
target_size = (256,256)
num_class = 2
num_test = 30

### Color Image

In [4]:
def Load_Color_Data(image_path, mask_path, image_prefix):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    
    for index,item in enumerate(image_name_arr):
        img = io.imread(item)
        img = np.reshape(img,img.shape + (1,))
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix))
        mask = np.reshape(mask,mask.shape + (1,))
        if img.max() > 1:
            img = img / 255
        if mask.max() > 1:
            mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
        image_arr.append(img)
        mask_arr.append(mask)

    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr, mask_arr

In [5]:
imgs_train, mask_train = Load_Color_Data(image_path, mask_path, image_prefix)

# Network

In [6]:
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
#    merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#    merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
#    merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
#    merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [7]:
model = unet()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

/home/jasonyu/anaconda3/envs/keras/lib/python3.5/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`


# Train

In [8]:
model_checkpoint = ModelCheckpoint('unet_cell_0804_1.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [9]:
model.fit(imgs_train, mask_train, batch_size=1, epochs=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

Train on 48 samples, validate on 12 samples
Epoch 1/10
48/48 [==============================] - 66s 1s/step - loss: 0.5700 - acc: 0.7508 - val_loss: 0.5199 - val_acc: 0.7823

Epoch 00001: loss improved from inf to 0.57003, saving model to unet_cell_0804_1.hdf5
Epoch 2/10
48/48 [==============================] - 55s 1s/step - loss: 0.4840 - acc: 0.7798 - val_loss: 0.4260 - val_acc: 0.7824

Epoch 00002: loss improved from 0.57003 to 0.48402, saving model to unet_cell_0804_1.hdf5
Epoch 3/10
48/48 [==============================] - 55s 1s/step - loss: 0.3869 - acc: 0.7798 - val_loss: 0.3471 - val_acc: 0.7824

Epoch 00003: loss improved from 0.48402 to 0.38695, saving model to unet_cell_0804_1.hdf5
Epoch 4/10
48/48 [==============================] - 55s 1s/step - loss: 0.3391 - acc: 0.7916 - val_loss: 0.3167 - val_acc: 0.8654

Epoch 00004: loss improved from 0.38695 to 0.33907, saving model to unet_cell_0804_1.hdf5
Epoch 5/10
48/48 [==============================] - 55s 1s/step - loss: 0.33

# Test

In [20]:
def Test_Color(test_path):
    test_name_arr = glob.glob(os.path.join(test_path,"*.png"))
    test_arr = []
    
    for index,item in enumerate(test_name_arr):
        test = io.imread(item)
        test = np.reshape(test,test.shape + (1,))
        test = test / 255
        test = trans.resize(test,target_size)
        test_arr.append(test)

    test_arr = np.array(test_arr)
    return test_arr, test_name_arr

In [21]:
model.load_weights('unet_cell_0804_1.hdf5')

In [22]:
imgs_test, test_filelist = Test_Color(test_path)

/home/jasonyu/anaconda3/envs/keras/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/jasonyu/anaconda3/envs/keras/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [23]:
imgs_test_results = model.predict(imgs_test, 1, verbose=1)

30/30 [==============================] - 9s 313ms/step


In [25]:
imgs_test_results[imgs_test_results > 0.5] = 1
imgs_test_results[imgs_test_results <= 0.5] = 0
imgs_test_results = np.reshape(imgs_test_results, (num_test,256,256))

In [26]:
for i in range(len(test_filelist)):
    file_path = test_filelist[i]
    (filepath,tempfilename) = os.path.split(file_path)
    (filename,extension) = os.path.splitext(tempfilename)
     #filename = int(filename)
    plt.imsave('%s%s.jpg'%(result_path,filename),imgs_test_results[i])
    print('No.%d: %s.jpg is done!'%(i,filename))

No.0: 27.jpg is done!
No.1: 12.jpg is done!
No.2: 8.jpg is done!
No.3: 6.jpg is done!
No.4: 26.jpg is done!
No.5: 29.jpg is done!
No.6: 19.jpg is done!
No.7: 25.jpg is done!
No.8: 28.jpg is done!
No.9: 11.jpg is done!
No.10: 4.jpg is done!
No.11: 20.jpg is done!
No.12: 22.jpg is done!
No.13: 5.jpg is done!
No.14: 10.jpg is done!
No.15: 17.jpg is done!
No.16: 2.jpg is done!
No.17: 14.jpg is done!
No.18: 7.jpg is done!
No.19: 13.jpg is done!
No.20: 3.jpg is done!
No.21: 1.jpg is done!
No.22: 23.jpg is done!
No.23: 24.jpg is done!
No.24: 9.jpg is done!
No.25: 0.jpg is done!
No.26: 15.jpg is done!
No.27: 21.jpg is done!
No.28: 16.jpg is done!
No.29: 18.jpg is done!
